Create and Test Team Ratings for 2023 Season - Round by Round

In [1]:
import pandas as pd
import re
import numpy as np
from scipy import stats
from sklearn import metrics

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

In [2]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats_v2.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

finals_round_map = {'F1':24,
                    'F2':25,
                    'F3':26,
                    'F4':27}
player_stats['Round_ID'] = player_stats['Round_ID'].astype(str)
player_stats['Round'] = (player_stats['Round_ID'].str[4:].replace(finals_round_map)).astype(int)
player_stats['Round_str'] = player_stats['Round_ID'].str[4:].replace(finals_round_map)
player_stats['Round_ID_num'] = (player_stats['Season'].astype(str) + player_stats['Round'].astype(str)).astype(int)

player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
28536,202324_WestCoast_Adelaide,West Coast,Ryan Maric,202324,Ryan_Maric,KEY_FORWARD,CD_I1029416,18,196,93,41,RIGHT,VIC,2023.0,2023.0,Drouin Vic)/Gippsland U18/Box Hill Hawks (VFL),1.0,other,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2004-09-06,71,0,0.0,NaN,0,0,NaN,NaN,NaN,0,NaN,2,NaN,NaN,NaN,NaN,77.8,9,49,NaN,NaN,NaN,0,2,0.0,1,0,NaN,3,0,NaN,NaN,NaN,0,NaN,1,NaN,NaN,NaN,6,NaN,5,0,NaN,114.0,0,NaN,NaN,1,NaN,3,NaN,0,NaN,0,2,0,0,10,2,8,NaN,NaN,Half Forward Flank Left,FINAL_TEAM,Half-Forward,Forward,NaN,NaN,2023,0.0000,-0.0587,0.0408,0.0525,-0.1360,0.1885,2.1304,0.9662,1.1643,0.2911,0.0000,0.4040,4.3952,Ryan Maric_2023,0,0.0000,West Coast,Adelaide,Adelaide,24,24,202324
28537,202324_WestCoast_Adelaide,West Coast,Sam Petrevski-Seton,202324,Sam_Petrevski-Seton,MEDIUM_FORWARD,CD_I994539,25,181,78,10,RIGHT,WA,2016.0,2017.0,Halls Creek (WA)/Claremont (WAFL)/Carlton,6.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-02-19,61,0,0.0,NaN,0,3,NaN,NaN,NaN,0,NaN,6,NaN,NaN,NaN,NaN,83.3,18,64,NaN,NaN,NaN,0,1,0.0,1,0,NaN,12,0,NaN,NaN,NaN,0,NaN,4,NaN,NaN,NaN,6,NaN,3,0,NaN,87.0,1,NaN,NaN,2,NaN,2,NaN,0,NaN,0,3,0,0,18,4,12,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023,0.0000,-0.2735,0.0312,-0.4014,-0.2771,-0.1243,-2.3624,-1.7040,-0.6584,-0.1613,0.1136,-0.0440,0.1591,Sam Petrevski-Seton_2023,0,0.0000,West Coast,Adelaide,Adelaide,24,24,202324
28538,202324_WestCoast_Adelaide,West Coast,Shannon Hurn,202324,Shannon_Hurn,MEDIUM_DEFENDER,CD_I240283,35,187,95,25,RIGHT,SA,2005.0,2006.0,Angaston (SA)/Central District (SANFL),13.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1987-09-04,83,0,0.0,NaN,0,2,NaN,NaN,NaN,0,NaN,2,NaN,NaN,NaN,NaN,95.5,22,89,NaN,NaN,NaN,0,1,0.0,0,0,NaN,4,0,NaN,NaN,NaN,0,NaN,4,NaN,NaN,NaN,18,NaN,6,0,NaN,506.0,3,NaN,NaN,7,NaN,2,NaN,0,NaN,1,2,0,1,12,2,10,NaN,NaN,Back Pocket Right,FINAL_TEAM,Back-Pocket,Back,NaN,NaN,2023,0.0000,-0.3356,0.0329,0.6958,0.1442,0.5515,1.3235,2.1248,-0.8013,-1.5501,0.1773,-0.4591,-1.6297,Shannon Hurn_2023,0,0.0000,West Coast,Adelaide,Adelaide,24,24,202324
28539,202324_WestCoast_Adelaide,West Coast,Tim Kelly,202324,Tim_Kelly,MIDFIELDER,CD_I295898,29,184,83,11,RIGHT,WA,2017.0,2018.0,Palmyra (WA)/South Fremantle (WAFL)/Geelong,24.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-26,84,0,2.0,NaN,4,5,NaN,NaN,NaN,0,NaN,15,NaN,NaN,Na

Pre-Season

In [3]:
def get_previous_round_id(round_id):
    
    finals_round_map = {'F1':24,
                        'F2':25,
                        'F3':26,
                        'F4':27}
    
    season = round_id[:4]
    if 'F' in round_id:
        round_num = finals_round_map[round_id[4:]]
    else:
        round_num = round_id[4:]
    
    previous_round_num = int(round_id[4:])-1
    previous_round_num = str(previous_round_num).zfill(2)
    if previous_round_num == 1:
        previous_season = int(season) - 1
    else:
        previous_season = season
    previous_round_id = str(previous_season) + str(previous_round_num)
    
    return previous_round_id

In [4]:
def get_score_standard_deviation(data):
    return data.groupby(['Match_ID', 'Team']).sum()['Score'].std()

In [5]:
def create_team_summary(player_stats, rounds = None):
    
    player_stats_copy = player_stats.copy()
    
    if rounds is not None:
        all_rounds = sorted(list(set(player_stats_copy['Round_ID'])))
        rating_rounds = all_rounds[-rounds:]
        player_stats_copy = player_stats_copy[player_stats_copy['Round_ID'].isin(rating_rounds)]
    
    team_for = player_stats_copy.groupby(['Team']).sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats_copy.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats_copy.groupby(['Match_ID', 'Team']).size().reset_index().groupby(['Team']).count()[['Match_ID']]
    team_games.columns = ['Games']
    team_summary = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_summary[col+"_pergame_F"] = team_summary[col+"_F"] / team_summary["Games"]
        team_summary[col+"_pergame_A"] = team_summary[col+"_A"] / team_summary["Games"]

        team_summary[col+"_diff"] = team_summary[col+"_F"] - team_summary[col+"_A"]
        team_summary[col+"_pergame_diff"] = team_summary[col+"_pergame_F"] - team_summary[col+"_pergame_A"]

    return team_summary

In [6]:
def get_teams(match_id):
    
    home_team = re.sub(r"\B([A-Z])", r" \1", match_id.split("_")[1])
    away_team = re.sub(r"\B([A-Z])", r" \1", match_id.split("_")[-1])

    return home_team, away_team

In [7]:
def calculate_match_projections(home_attack_rating, home_defence_rating, away_attack_rating, away_defence_rating):
    
    home_projected_score = (home_attack_rating + away_defence_rating)*0.5
    away_projected_score = (home_defence_rating + away_attack_rating)*0.5
    projected_margin = home_projected_score - away_projected_score
    
    return home_projected_score, away_projected_score, projected_margin

In [8]:
def get_match_ratings(team_summary, home_team, away_team):
    
    match_ratings_dict = {'home_attack_rating':team_summary.loc[home_team]['xScore_pergame_F'],
                          'home_defence_rating':team_summary.loc[home_team]['xScore_pergame_A'],
                          'away_attack_rating':team_summary.loc[away_team]['xScore_pergame_F'],
                          'away_defence_rating':team_summary.loc[away_team]['xScore_pergame_A']}
    
    return match_ratings_dict

In [9]:
def calculate_projected_probabilities(home_projected_score, away_projected_score, score_std=25):
    
    projected_margin = home_projected_score - away_projected_score
    projected_margin_distribution = stats.norm(loc = projected_margin, scale = score_std)
    
    away_team_win_probability = projected_margin_distribution.cdf(0)
    draw_probability = projected_margin_distribution.pdf(0)
    home_team_win_probability = 1 - away_team_win_probability - draw_probability
    
    return home_team_win_probability, draw_probability, away_team_win_probability
    

In [10]:
def get_match_prediction_metrics(projected_margin, margin):
    
    mae = abs(projected_margin - margin)
    mse = ((projected_margin)**2 + (margin)**2)
    rmse = ((projected_margin)**2 + (margin)**2)**0.5
    
    return mae, mse, rmse

In [11]:
def get_match_result(data, match_id):
    
    home_team, away_team = get_teams(match_id)
    match_player_stats = data[data['Match_ID'] == match_id]
    match_score = match_player_stats.groupby('Team').sum()[['Score', 'xScore']]
    home_score, home_xscore = match_score.loc[home_team]['Score'], match_score.loc[home_team]['xScore']
    away_score, away_xscore = match_score.loc[away_team]['Score'], match_score.loc[away_team]['xScore']
    margin, xmargin = home_score - away_score, home_xscore - away_xscore
    
    return home_score, away_score, margin

In [12]:
def calculate_match_projection_error(player_stats, match_id, match_ratings_dict, score_std = 25):
    
    home_projected_score, away_projected_score, projected_margin = calculate_match_projections(match_ratings_dict['home_attack_rating'], 
                                                                             match_ratings_dict['home_defence_rating'], 
                                                                             match_ratings_dict['away_attack_rating'], 
                                                                             match_ratings_dict['away_defence_rating'])
    
    home_team_win_probability, draw_probability, away_team_win_probability = calculate_projected_probabilities(home_projected_score, away_projected_score, score_std)
    home_score, away_score, margin = get_match_result(player_stats, match_id)
    
    mae, mse, rmse = get_match_prediction_metrics(projected_margin, margin)
    
    return mae, mse, rmse

In [13]:
def get_latest_rounds(round_id, player_stats, rounds):
    
    all_rounds = sorted(list(set(player_stats['Round_ID'])))
    rating_rounds = all_rounds[all_rounds.index(round_id)-rounds:all_rounds.index(round_id)]
    player_stats_latest = player_stats[player_stats['Round_ID'].isin(rating_rounds)]
    
    return player_stats_latest

In [14]:
def update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae):
    
    previous_round_id = get_previous_round_id(round_id)
    player_stats_latest = get_latest_rounds(round_id, player_stats, rounds)
    team_summary = create_team_summary(player_stats_latest, rounds = rounds)
    team_summary = team_summary[['xScore_pergame_F', 'xScore_pergame_A', 'xScore_pergame_diff']]

    round_projection_dict[round_id] = create_round_match_projections(player_stats, team_summary, round_id)
    round_mae[round_id] = np.mean(np.array(list({v['MAE'] for (k, v) in round_projection_dict[round_id].items()})))
    print("Round MAE: {}".format(round_mae[round_id]))
    print("Overall MAE: {}".format(np.mean(np.array(list(round_mae.values())))))
    return team_summary.sort_values('xScore_pergame_diff', ascending = False)

In [15]:
def create_round_match_projections(player_stats, team_summary, round_id):
    
    player_stats_round = player_stats[player_stats['Round_ID'] == round_id]
    match_id_list = list(player_stats_round['Match_ID'].unique())

    match_projection_dict = {}
    for match_id in match_id_list:
        home_team, away_team = get_teams(match_id)
        match_ratings_dict = get_match_ratings(team_summary, home_team, away_team)
        home_projected_score, away_projected_score, projected_margin = calculate_match_projections(match_ratings_dict['home_attack_rating'], 
                                                                                                    match_ratings_dict['home_defence_rating'], 
                                                                                                    match_ratings_dict['away_attack_rating'], 
                                                                                                    match_ratings_dict['away_defence_rating'])
        home_team_win_probability, draw_probability, away_team_win_probability = calculate_projected_probabilities(home_projected_score, away_projected_score, score_std)
        home_score, away_score, margin = get_match_result(player_stats_round, match_id)
        mae, mse, rmse = calculate_match_projection_error(player_stats_round, match_id, match_ratings_dict, score_std = 25)

        match_projection_dict[match_id] = {}
        if home_projected_score > away_projected_score:
            match_projection_dict[match_id]['tip'] = home_team
            match_projection_dict[match_id]['projected_prob'] = home_team_win_probability
        else:
            match_projection_dict[match_id]['tip'] = away_team
            match_projection_dict[match_id]['projected_prob'] = away_team_win_probability
        match_projection_dict[match_id]['projected_margin'] = abs(projected_margin)
        if home_score > away_score:
            match_projection_dict[match_id]['result'] = home_team
        else:
            match_projection_dict[match_id]['result'] = away_team
        match_projection_dict[match_id]['actual_margin'] = abs(margin)   
        match_projection_dict[match_id]['MAE'] = mae
    
    return match_projection_dict
    

In [16]:
score_std = get_score_standard_deviation(player_stats)

In [17]:
round_projection_dict = {}
round_mae = {}
rounds = 10

202301

In [18]:
round_id = '202301'
player_stats_latest = player_stats[player_stats['Season'] == 2022]
team_summary = create_team_summary(player_stats_latest, rounds = rounds)
team_summary = team_summary[['xScore_pergame_F', 'xScore_pergame_A', 'xScore_pergame_diff']]

round_projection_dict[round_id] = create_round_match_projections(player_stats, team_summary, round_id)
round_mae[round_id] = np.mean(np.array(list({v['MAE'] for (k, v) in round_projection_dict[round_id].items()})))
print("Round MAE: {}".format(round_mae[round_id]))
print("Overall MAE: {}".format(np.mean(np.array(list(round_mae.values())))))
team_summary.sort_values('xScore_pergame_diff', ascending = False)

Round MAE: 31.402605292587793
Overall MAE: 31.402605292587793


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,98.9063,58.7470,40.1593
Richmond,99.8391,77.7397,22.0994
Sydney,91.2116,73.4233,17.7883
Port Adelaide,91.1296,80.8188,10.3107
Melbourne,84.8261,74.8366,9.9895
Western Bulldogs,80.2241,73.7809,6.4432
Brisbane Lions,85.9923,83.7165,2.2759
Fremantle,73.8515,75.6771,-1.8257
Carlton,78.7282,81.4170,-2.6887
St Kilda,74.4954,81.1609,-6.6655


202302

In [19]:
round_id = '202302'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 35.666881069044
Overall MAE: 33.534743180815894


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,98.6147,64.5397,34.0750
Port Adelaide,100.7617,76.2723,24.4894
Sydney,94.1329,73.0418,21.0911
Richmond,96.0286,75.2991,20.7295
Melbourne,87.9351,73.0550,14.8801
Western Bulldogs,79.4495,78.9416,0.5079
Fremantle,74.6111,74.6585,-0.0473
Carlton,79.4761,79.7711,-0.2950
Collingwood,77.0760,81.9747,-4.8987
St Kilda,74.8258,81.5405,-6.7147


202303

In [20]:
round_id = '202303'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 27.32547382224668
Overall MAE: 31.464986727959488


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,96.5536,64.7285,31.8251
Sydney,95.0324,72.6641,22.3682
Richmond,99.3067,79.4361,19.8706
Port Adelaide,96.6369,79.0025,17.6344
Melbourne,84.6959,72.2752,12.4207
Fremantle,75.7961,73.9485,1.8475
Collingwood,81.1088,80.3513,0.7575
Western Bulldogs,73.6771,77.2304,-3.5533
Carlton,77.2207,81.7039,-4.4833
St Kilda,76.2476,80.8263,-4.5788


202304

In [21]:
round_id = '202304'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 31.80961631404953
Overall MAE: 31.551144124482


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,94.0423,66.1898,27.8525
Richmond,92.5534,75.4413,17.1121
Port Adelaide,98.3566,83.1124,15.2442
Sydney,91.7930,79.6096,12.1834
Melbourne,87.0650,76.5246,10.5404
Fremantle,81.2517,73.3189,7.9328
Collingwood,80.9288,78.3478,2.5810
Carlton,80.0209,77.6718,2.3492
Western Bulldogs,73.2657,72.9528,0.3128
Greater Western Sydney,82.3593,85.3604,-3.0011


202305

In [22]:
round_id = '202305'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 28.115834261643712
Overall MAE: 30.86408215191434


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,95.8594,66.5469,29.3125
Port Adelaide,98.6028,78.0287,20.5741
Melbourne,89.3564,75.1430,14.2133
Sydney,87.8442,77.5839,10.2603
Richmond,87.0537,78.2537,8.8000
Carlton,82.9471,75.4507,7.4963
St Kilda,83.0272,76.5104,6.5168
Fremantle,80.5905,74.2191,6.3714
Collingwood,81.4597,79.9708,1.4889
Western Bulldogs,74.3630,73.0150,1.3480


202306

In [23]:
round_id = '202306'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 29.024342550070017
Overall MAE: 30.557458884940285


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,96.0205,69.1430,26.8775
Port Adelaide,92.1753,77.8385,14.3368
Melbourne,88.8756,78.5226,10.3530
Sydney,90.9195,81.7796,9.1398
St Kilda,81.7763,73.4617,8.3146
Essendon,97.7858,89.5445,8.2413
Collingwood,83.3506,77.7515,5.5991
Adelaide,90.7906,87.3715,3.4191
Fremantle,82.1473,79.2832,2.8641
Richmond,85.1114,85.0040,0.1073


202307

In [24]:
round_id = '202307'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 32.049244358723655
Overall MAE: 30.770571095480765


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,94.8594,70.4912,24.3682
Essendon,96.8705,82.6825,14.1879
Adelaide,94.9260,82.3554,12.5706
St Kilda,82.1273,70.1873,11.9400
Port Adelaide,90.6416,79.2592,11.3824
Collingwood,85.4169,76.3061,9.1108
Melbourne,88.2604,81.3130,6.9474
Brisbane Lions,88.7141,83.6817,5.0324
Sydney,84.6666,84.9699,-0.3032
Western Bulldogs,75.7156,78.5603,-2.8447


202308

In [25]:
round_id = '202308'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 20.47108931787254
Overall MAE: 29.483135873279743


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,97.8527,71.8660,25.9867
Melbourne,92.3359,77.6703,14.6655
Adelaide,92.0196,79.2095,12.8101
Carlton,87.6457,77.0532,10.5925
St Kilda,82.4798,72.0332,10.4466
Port Adelaide,89.5655,80.0214,9.5440
Collingwood,84.0790,75.1478,8.9312
Brisbane Lions,90.0268,81.5947,8.4321
Essendon,95.1386,86.8629,8.2758
Sydney,87.5796,86.3418,1.2378


202309

In [26]:
round_id = '202309'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 39.320484062376764
Overall MAE: 30.576174560957188


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,96.7026,71.4846,25.2180
Port Adelaide,91.9045,78.3172,13.5872
Melbourne,91.1540,78.2562,12.8978
Brisbane Lions,91.2771,79.2533,12.0238
St Kilda,81.9077,71.1292,10.7784
Adelaide,89.0237,80.1023,8.9214
Collingwood,83.4190,76.1882,7.2308
Carlton,85.1011,78.5599,6.5413
Essendon,91.5448,89.5397,2.0051
Sydney,85.8968,85.7500,0.1467


202310

In [27]:
round_id = '202310'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 31.011636098881507
Overall MAE: 30.619720714749622


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Geelong,95.5506,73.4511,22.0996
Brisbane Lions,96.0255,75.6949,20.3306
Port Adelaide,93.2109,75.6032,17.6077
Melbourne,91.0735,74.8172,16.2563
Collingwood,86.2535,74.1643,12.0893
Adelaide,90.8262,79.6650,11.1612
Gold Coast,87.2620,80.6109,6.6511
St Kilda,81.2698,74.9200,6.3498
Carlton,82.6825,77.2790,5.4035
Fremantle,84.4265,82.5777,1.8487


202311

In [28]:
round_id = '202311'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 19.084900172415328
Overall MAE: 29.5711006654465


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,96.8511,74.4649,22.3862
Port Adelaide,93.9910,75.7690,18.2221
Geelong,93.1771,78.9451,14.2321
Collingwood,86.5088,73.2840,13.2249
Melbourne,89.6923,77.4367,12.2555
St Kilda,83.0172,74.1959,8.8212
Western Bulldogs,77.9364,72.5416,5.3949
Adelaide,87.1413,82.1952,4.9460
Sydney,88.2129,84.9385,3.2743
Fremantle,85.7467,82.5326,3.2142


202312

In [29]:
round_id = '202312'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 20.918241420496795
Overall MAE: 28.850029061700692


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,100.1918,69.4360,30.7557
Geelong,92.2238,77.2032,15.0206
Port Adelaide,90.4418,77.5930,12.8488
Collingwood,84.8238,72.0842,12.7396
Western Bulldogs,80.3018,70.1833,10.1185
Melbourne,87.6973,77.8091,9.8882
St Kilda,84.5814,78.0817,6.4997
Adelaide,85.7495,79.9327,5.8168
Gold Coast,86.8285,81.1830,5.6455
Carlton,81.6408,77.5223,4.1185


202313

In [30]:
round_id = '202313'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 29.2442510026805
Overall MAE: 28.88035382639145


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,101.7532,69.8074,31.9459
Port Adelaide,96.7303,76.4924,20.2379
Geelong,93.3116,77.2612,16.0504
Melbourne,89.5877,73.6859,15.9018
Collingwood,84.8747,71.0258,13.8489
Western Bulldogs,84.3871,70.5633,13.8238
Gold Coast,87.4495,78.9857,8.4638
Adelaide,85.4381,80.9882,4.4500
St Kilda,85.0872,80.9830,4.1042
Fremantle,87.2872,86.5319,0.7553


202314

In [31]:
round_id = '202314'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 33.7616647988797
Overall MAE: 29.229018895854892


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,103.0238,71.4241,31.5997
Port Adelaide,98.3222,73.3442,24.9780
Geelong,95.9727,76.8222,19.1505
Melbourne,87.7182,71.1250,16.5933
Adelaide,89.3383,77.9155,11.4228
Western Bulldogs,84.9832,74.2552,10.7279
Collingwood,83.5072,74.4139,9.0933
Gold Coast,88.1426,80.0550,8.0876
St Kilda,83.9103,79.0224,4.8879
Fremantle,84.8887,85.3661,-0.4774


202315

In [32]:
round_id = '202315'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 44.18979844321796
Overall MAE: 30.226404199012432


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,102.7416,69.1831,33.5585
Port Adelaide,102.4413,71.8776,30.5637
Melbourne,84.7903,71.2465,13.5438
Collingwood,83.2456,70.4215,12.8241
Western Bulldogs,86.1808,73.9502,12.2307
Adelaide,89.1659,78.6533,10.5126
Geelong,90.5662,83.1839,7.3824
Gold Coast,87.2490,81.8750,5.3740
Carlton,78.9882,77.7273,1.2609
St Kilda,80.7971,80.4234,0.3737


202316

In [33]:
round_id = '202316'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 37.79973319339761
Overall MAE: 30.69973726116151


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Port Adelaide,104.5827,72.9619,31.6208
Brisbane Lions,94.8600,70.0367,24.8233
Melbourne,84.9112,68.0849,16.8263
Western Bulldogs,88.8544,72.9259,15.9285
Gold Coast,89.6196,76.0819,13.5378
Collingwood,85.1132,72.2139,12.8993
Carlton,81.1503,74.3735,6.7768
Adelaide,86.1884,81.6784,4.5100
Geelong,86.2181,82.8259,3.3922
Richmond,82.8255,81.3936,1.4319


202317

In [34]:
round_id = '202317'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 23.892696292042064
Overall MAE: 30.299323086507425


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,97.1629,67.1357,30.0271
Port Adelaide,100.4250,74.2477,26.1773
Melbourne,84.4057,63.6614,20.7442
Collingwood,89.8933,70.8299,19.0634
Carlton,83.3147,71.1199,12.1948
Sydney,90.1291,78.2476,11.8815
Adelaide,91.0813,79.4880,11.5932
Western Bulldogs,84.6840,74.3776,10.3065
St Kilda,84.1101,79.6526,4.4575
Fremantle,83.1493,80.6247,2.5246


202318

In [35]:
round_id = '202318'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 30.033418298236192
Overall MAE: 30.28455059827013


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,98.7310,63.5811,35.1499
Port Adelaide,102.6532,72.1975,30.4557
Collingwood,91.9604,71.2080,20.7524
Melbourne,79.4995,65.9768,13.5227
Western Bulldogs,83.9791,76.1224,7.8567
Adelaide,90.9366,84.2684,6.6681
Sydney,86.1721,80.4006,5.7715
Carlton,79.0064,73.3157,5.6907
St Kilda,82.2614,77.0942,5.1672
Fremantle,82.8862,79.9214,2.9648


202319

In [36]:
round_id = '202319'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 16.408088564130594
Overall MAE: 29.554210491210153


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,98.8101,64.6140,34.1961
Port Adelaide,100.4860,77.3805,23.1054
Collingwood,94.2644,71.9542,22.3101
Melbourne,79.2417,66.7855,12.4562
Carlton,84.0894,73.2786,10.8107
Sydney,87.5091,80.0189,7.4902
Adelaide,90.2868,82.8986,7.3882
Western Bulldogs,84.5989,77.2548,7.3441
Geelong,86.2394,82.5924,3.6470
Greater Western Sydney,82.9218,81.1845,1.7373


202320

In [37]:
round_id = '202320'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 25.840747960228704
Overall MAE: 29.36853736466108


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,94.6481,64.4483,30.1997
Port Adelaide,98.3445,77.6816,20.6629
Carlton,91.5799,72.6655,18.9145
Collingwood,88.7442,73.3460,15.3982
Sydney,89.0177,76.3374,12.6803
Greater Western Sydney,86.9490,74.9270,12.0219
Western Bulldogs,85.9281,75.9804,9.9477
Melbourne,80.3072,71.7298,8.5773
St Kilda,79.4332,73.7658,5.6675
Adelaide,88.7932,85.5488,3.2444


202321

In [38]:
round_id = '202321'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 22.638535881611116
Overall MAE: 29.048061103563462


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,91.3100,68.3774,22.9327
Carlton,93.6353,73.3914,20.2439
Greater Western Sydney,89.2368,73.0387,16.1981
Melbourne,85.2852,69.9382,15.3471
Collingwood,89.4701,75.4014,14.0687
Port Adelaide,95.0309,81.3773,13.6536
Adelaide,95.0760,81.7957,13.2803
Sydney,89.4536,77.1315,12.3222
St Kilda,79.9225,75.0756,4.8469
Geelong,82.9493,79.3729,3.5763


202322

In [39]:
round_id = '202322'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 23.152707868927937
Overall MAE: 28.780090501989118


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Brisbane Lions,90.3510,67.2116,23.1394
Carlton,93.1784,71.1014,22.0770
Adelaide,97.6408,79.1255,18.5153
Greater Western Sydney,88.3968,72.6451,15.7517
Melbourne,85.9147,71.1597,14.7550
Sydney,89.8169,78.8830,10.9339
Port Adelaide,94.4570,84.8858,9.5712
Western Bulldogs,88.5820,79.4379,9.1441
Collingwood,86.3901,78.1123,8.2778
St Kilda,75.8817,69.9249,5.9568


202323

In [40]:
round_id = '202323'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 30.45762127434098
Overall MAE: 28.85302662252616


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Carlton,94.3888,68.2654,26.1234
Adelaide,99.4127,77.9999,21.4128
Brisbane Lions,90.2940,70.5124,19.7816
Greater Western Sydney,86.9000,72.4942,14.4058
Sydney,90.2768,78.9524,11.3244
Melbourne,83.0787,72.3701,10.7086
Port Adelaide,91.9709,82.6424,9.3285
Western Bulldogs,86.2583,79.0184,7.2399
St Kilda,77.7723,70.6666,7.1057
Geelong,86.2608,81.0155,5.2453


202324

In [41]:
round_id = '202323'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 30.45762127434098
Overall MAE: 28.85302662252616


,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Carlton,94.3888,68.2654,26.1234
Adelaide,99.4127,77.9999,21.4128
Brisbane Lions,90.2940,70.5124,19.7816
Greater Western Sydney,86.9000,72.4942,14.4058
Sydney,90.2768,78.9524,11.3244
Melbourne,83.0787,72.3701,10.7086
Port Adelaide,91.9709,82.6424,9.3285
Western Bulldogs,86.2583,79.0184,7.2399
St Kilda,77.7723,70.6666,7.1057
Geelong,86.2608,81.0155,5.2453
